## EE2100 TUTORIAL 7 : Q1
### RAJ PATIL : CS18BTECH11039

In [266]:
import numpy as np
import numpy.linalg as nla
import random
from typing import Callable
import pandas as pd
from numpy.linalg import qr

## Q1

### Establishing basic algo tester

In [267]:
np.random.seed(0)

In [268]:
def test_GS(gs_algo: Callable[[np.array,int],tuple] ,num_trials: int,ret_errs= False) -> bool:
    errs = []
    for t in range(num_trials):
        m,n = random.choice(range(2,6)),random.choice(range(2,6))
        A = np.random.randn(m,n)
        Q,R = gs_algo(A)
        errs.append(nla.norm(np.matmul(Q,R) - A,"fro"))
    print(f"mean Frobenius error: {np.array(errs).mean()}")
    if ret_errs:
        return errs

### Classical Gram-Schmidt 

In [269]:
def classical_gram_schmidt(A: np.array,eps = 1e-4) -> (np.array,np.array):
    m,n = A.shape
    Q = np.zeros((m,n))
    R = np.zeros((n,n))
    for j in range(n):
        cache = np.zeros(m)
        for k in range(j):
            R[k,j] = np.inner(A[:,j],Q[:,k])
            cache += R[k,j]
        v_j_tilde = A[:,j] - cache
        L2_v_j_tilde = R[j,j] = nla.norm(v_j_tilde,2)
        if L2_v_j_tilde < eps:
            Q[:,j] = np.zeros(m)
        else:
            Q[:,j] = v_j_tilde/L2_v_j_tilde
    return Q,R

In [270]:
test_GS(classical_gram_schmidt,1000)

mean Frobenius error: 4.109175083843623


### Modified Gram-Schmidt

In [271]:
def modified_gram_schmidt(A: np.array,eps = 1e-4) -> (np.array,np.array):
    m,n = A.shape
    Q = A.copy()
    R = np.zeros((n,n))
    for j in range(n):
        R[j,j] = nla.norm(Q[:,j])
        if R[j,j] < eps:
            Q[:,j] = np.zeros(m)
        else:
            Q[:,j] = Q[:,j]/R[j,j]
        for k in range(j+1,n):
            R[j,k] = np.inner(Q[:,k],Q[:,j])
            Q[:,k] = Q[:,k] - R[j,k]*Q[:,j]
    return Q,R

In [272]:
test_GS(modified_gram_schmidt,1000)

mean Frobenius error: 7.927147733285168e-16


## Final comparison (Q1)

In [273]:
from itertools import product
algo = ["CGS","MGS","NUMPY_QR"]
inputs = ["random","random singular","quasi singular","Identity jittered Hilbert"]
metric = ["||A - Q R||","||Q.T Q - I||"]

results = pd.concat([pd.DataFrame(data= list(product(inputs,metric)),columns=["Input","Metric"]),pd.DataFrame(columns=algo)])
results = results.set_index(keys=["Input","Metric"])

In [274]:
def test_gs(A: np.array,gs_algo: Callable[[np.array,int],tuple],descr: str):
    Q,R = gs_algo(A)
    return nla.norm(A - np.matmul(Q,R),'fro'),nla.norm(np.matmul(Q.T,Q) - np.identity(R.shape[0]),'fro')

In [275]:
def gen_hilbert(size:int) -> np.array:
    H = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            H[i,j] = 1/(i + j + 1) # zero indexed
    return H

In [276]:
def compare_test(seed: int, size:int, results: pd.DataFrame):
    np.random.seed(seed)
    
    algo_hash = {"CGS":classical_gram_schmidt,
                 "MGS":modified_gram_schmidt,
                 "NUMPY_QR":nla.qr}
    
    rand_square = np.random.randn(size,size)
    rand_skinny = np.random.randn(size,np.random.choice(range(1,size)))
    rand_singular = np.matmul(rand_skinny,rand_skinny.T)
    jitter = np.identity(size)*1e-4
    mat_hash = {"random" : rand_square,
                "random singular" : rand_singular,
                "quasi singular" : jitter + rand_singular,
                "Identity jittered Hilbert": jitter + gen_hilbert(size)}
    
    for algo in algo_hash.keys():
        for mat in mat_hash.keys():
                results.loc[mat,algo] = test_gs(mat_hash[mat],algo_hash[algo],f'input: {mat}, algorithm {algo}')

    return results

In [277]:
compare_test(1729,3,results)

CGS          MGS     NUMPY_QR
Input                     Metric                                           
random                    ||A - Q R||     6.05708  1.11022e-16  1.21619e-15
                          ||Q.T Q - I||   1.90277  4.81935e-16  6.14257e-16
random singular           ||A - Q R||    0.545402  7.87346e-17  7.86772e-17
                          ||Q.T Q - I||   1.81391      1.41421  5.49956e-16
quasi singular            ||A - Q R||    0.545492            0  1.60094e-16
                          ||Q.T Q - I||   1.81317  3.63673e-14  6.21086e-16
Identity jittered Hilbert ||A - Q R||    0.901018  5.55112e-17  1.38778e-16
                          ||Q.T Q - I||   2.06521  2.15293e-14  5.86758e-16